# DATA301 Project Phase 2

This is my notebook for phase 2 of my project for DATA301.

In it I will go through all the stages of loading in the data, cleaning and preprocessing it, examining the features, and finally producing the best model I can for my fitted data

### Part 1 - Loading

In this part I will load all the required libraries, as well as the required datasets

Loading the libraries

In [110]:
import numpy as np
import pandas as pd
import sklearn.linear_model

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

Now we will load all the datasets that we want to use
1. Body Measures - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.htm
2. Blood Pressure - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPX_J.htm
3. Cardiovasular Health - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/CDQ_J.htm
4. Demographics - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm4.
5. Medical Conditions - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/MCQ_J.htm

In [111]:
body_measures = pd.read_sas('data\BMX_J.XPT')
blood_pressure = pd.read_sas('data\BPX_J.XPT')
cardio_health = pd.read_sas('data\CDQ_J.XPT')
demographics = pd.read_sas('data\DEMO_J.XPT')
medical_conditions = pd.read_sas('data\MCQ_J.XPT')

Now we need to drop several columns.

#### Body Measures Dataset

For our first dataset, the body measures one, there are only 4 columns that we are interested in from looking at the documentation (https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.htm), which would be the SEQN, weight, height, and BMI, and the rest seem unneccesary so we will exclude them.

In [112]:
body_measures_short = body_measures[['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI']]
body_measures_short.columns = ['SEQN', 'Weight', 'Height', 'BMI']
print(body_measures_short.head())

print('Total number of respondents')
print(len(body_measures_short))
print('% NaN occurrences in Columns:')
print(100 * body_measures_short.isnull().sum(axis = 0)/len(body_measures_short))

      SEQN  Weight  Height   BMI
0  93703.0    13.7    88.6  17.5
1  93704.0    13.9    94.2  15.7
2  93705.0    79.5   158.3  31.7
3  93706.0    66.3   175.7  21.5
4  93707.0    45.4   158.4  18.1
Total number of respondents
8704
% NaN occurrences in Columns:
SEQN      0.000000
Weight    1.424632
Height    7.904412
BMI       8.030790
dtype: float64


We can see that of the columns we have selected, they have very few NA values in them which means that we can be happy using them for now

#### Blood Pressure Dataset

For our second dataset, the blood pressure one, there are several unneccessary columns. From reading the documentation (https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPX_J.htm), we can see that there are columns about cuff size, and which arm the measurement was taken on, but we are only interested in the recorded blood pressure and the resting heart rate. Since the blood pressure was measured 4 consecutive times, we are going to take an average of the four as our resulting blood pressures, and keep these along with their 60 second heart rate. But first, we need to check if any of these columns have high occurrences of NAs, as they will affect the output

In [113]:
print('Total number of respondents')
print(len(blood_pressure))
print('% NaN occurrences in Columns:')
print(100 * blood_pressure.isnull().sum(axis = 0)/len(blood_pressure))

Total number of respondents
8704
% NaN occurrences in Columns:
SEQN         0.000000
PEASCCT1    94.898897
BPXCHR      82.318474
BPAARM      21.679688
BPACSZ      21.737132
BPXPLS      22.541360
BPXPULS      4.859835
BPXPTY      22.541360
BPXML1      22.667739
BPXSY1      27.596507
BPXDI1      27.596507
BPAEN1      27.596507
BPXSY2      24.597886
BPXDI2      24.597886
BPAEN2      24.597886
BPXSY3      24.885110
BPXDI3      24.885110
BPAEN3      24.885110
BPXSY4      93.612132
BPXDI4      93.612132
BPAEN4      93.612132
dtype: float64


Most of the columns that we want to use have about 25% of reponses that are NAs, which is okay for our purposes. However, the fourth reading has 93% NA responses, so we will exclude it from our calculations and just use the first 3 readings of the blood pressures

In [114]:
blood_pressure['diastolic_avg'] = blood_pressure[['BPXDI1', 'BPXDI2', 'BPXDI3']].mean(axis=1, skipna=False)
blood_pressure['systolic_avg'] = blood_pressure[['BPXSY1', 'BPXSY2', 'BPXSY3']].mean(axis=1, skipna=False)
blood_pressure['pulse_rate_BPM'] = blood_pressure['BPXPLS']
blood_pressure_short = blood_pressure[['SEQN', 'diastolic_avg', 'systolic_avg', 'pulse_rate_BPM']]
print(blood_pressure_short.head())
print(len(blood_pressure_short[blood_pressure_short.isna().any(axis=1)]))
print(len(blood_pressure_short))

      SEQN  diastolic_avg  systolic_avg  pulse_rate_BPM
0  93703.0            NaN           NaN             NaN
1  93704.0            NaN           NaN             NaN
2  93705.0            NaN           NaN            52.0
3  93706.0      73.333333    111.333333            82.0
4  93707.0      47.333333    128.000000           100.0
2627
8704


From this we can see that there are now around 20% of responses that have an NA in one or more of these columns, which is an acceptable number, and we will deal with these later.

#### Cardiovascular Health Dataset

Next we are going to take a look at the cardiovascular health dataset, and see if there are any columns which need to be dropped. Referring to the documentation, https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/CDQ_J.htm, there are a lot of columns which seem useful, so we will first check the percentage of NA responses for each column

In [115]:
print('Total number of respondents')
print(len(cardio_health))
print('% NaN occurrences in Columns:')
print(100 * cardio_health.isnull().sum(axis = 0)/len(cardio_health))

Total number of respondents
3882
% NaN occurrences in Columns:
SEQN        0.000000
CDQ001      0.000000
CDQ002     70.736734
CDQ003     90.288511
CDQ004     91.164348
CDQ005     91.859866
CDQ006     93.122102
CDQ009A    99.613601
CDQ009B    98.995363
CDQ009C    99.665121
CDQ009D    96.548171
CDQ009E    98.402885
CDQ009F    97.758887
CDQ009G    99.639361
CDQ009H    99.845440
CDQ008     70.736734
CDQ010      0.000000
dtype: float64


This dataset seems very empty, as there are only 3 columns that have more than 30% of respondents answering, so I do not think that this dataset is very good to use, as trying to fill in the missing values will be too hard as there are too many of them for it to be useful, so we will no longer work with this dataset

#### Demographics Dataset

Thirdly we take a look at the demographics dataset. According to the documentation, https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm, this contains questions about peoples lives that are more general than the strictly medical data we have looked at so far. There are also a lot more columns in this dataset than the others, so we will see if there are any columns that we can drop based on them having too many null values

In [116]:
print('Total number of respondents')
print(len(demographics))
print('% NaN occurrences in Columns:')
print(100 * demographics.isnull().sum(axis = 0)/len(demographics))

Total number of respondents
9254
% NaN occurrences in Columns:
SEQN         0.000000
SDDSRVYR     0.000000
RIDSTATR     0.000000
RIAGENDR     0.000000
RIDAGEYR     0.000000
RIDAGEMN    93.548736
RIDRETH1     0.000000
RIDRETH3     0.000000
RIDEXMON     5.943376
RIDEXAGM    62.902529
DMQMILIZ    35.119948
DMQADFC     93.937757
DMDBORN4     0.000000
DMDCITZN     0.032418
DMDYRSUS    78.949643
DMDEDUC3    75.081046
DMDEDUC2    39.820618
DMDMARTL    39.820618
RIDEXPRG    88.005187
SIALANG      0.000000
SIAPROXY     0.000000
SIAINTRP     0.000000
FIALANG      5.122109
FIAPROXY     5.122109
FIAINTRP     5.122109
MIALANG     27.771774
MIAPROXY    27.771774
MIAINTRP    27.771774
AIALANGA    46.217852
DMDHHSIZ     0.000000
DMDFMSIZ     0.000000
DMDHHSZA     0.000000
DMDHHSZB     0.000000
DMDHHSZE     0.000000
DMDHRGND     0.000000
DMDHRAGZ     0.000000
DMDHREDZ     5.295008
DMDHRMAZ     2.063972
DMDHSEDZ    48.660039
WTINT2YR     0.000000
WTMEC2YR     0.000000
SDMVPSU      0.000000
SDMVSTRA     

Most of the columns have very few NA values, so we will go ahead and drop all those that have high NA responses

In [117]:
demographics_short = demographics.copy()
demographics_short.drop(['RIDAGEMN', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHSEDZ'], axis=1, inplace=True)
print(demographics_short.columns)

Index(['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1',
       'RIDRETH3', 'RIDEXMON', 'DMDBORN4', 'DMDCITZN', 'SIALANG', 'SIAPROXY',
       'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP', 'DMDHHSIZ', 'DMDFMSIZ',
       'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ',
       'DMDHRMAZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA', 'INDHHIN2',
       'INDFMIN2', 'INDFMPIR'],
      dtype='object')


Next we will select all the columns that we deem relevant. Looking through the dataset, the ones which we do not want to keep are ones that are less about the person, like the data release cycle, and the time period over which the questionaire was conducted, and other irrelevant information such as the information about the household reference person. We do this to narrow down the number of features so that the classification does not become overfitted

In [118]:
demographics_short2 = demographics_short.copy()
demographics_short2.drop(['SDDSRVYR', 'RIDSTATR', 'RIDRETH3', 'RIDEXMON', 'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP', 'DMDHHSIZ', 'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ', 'DMDHRMAZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA'], axis=1, inplace=True)
print(demographics_short2.head())
print(demographics_short2.columns)

      SEQN  RIAGENDR  RIDAGEYR  RIDRETH1  DMDBORN4  DMDCITZN  DMDFMSIZ  \
0  93703.0       2.0       2.0       5.0       1.0       1.0       5.0   
1  93704.0       1.0       2.0       3.0       1.0       1.0       4.0   
2  93705.0       2.0      66.0       4.0       1.0       1.0       1.0   
3  93706.0       1.0      18.0       5.0       1.0       1.0       5.0   
4  93707.0       1.0      13.0       5.0       1.0       1.0       7.0   

   INDHHIN2  INDFMIN2  INDFMPIR  
0      15.0      15.0      5.00  
1      15.0      15.0      5.00  
2       3.0       3.0      0.82  
3       NaN       NaN       NaN  
4      10.0      10.0      1.88  
Index(['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN4', 'DMDCITZN',
       'DMDFMSIZ', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR'],
      dtype='object')


Finally we will rename the variables in this dataset too so that they are clearer to understand

In [119]:
demographics_short2.columns = ['SEQN', 'Gender', 'Age', 'Race', 'CountryOfBirth', 'CitizenshipStatus', 'FamilySize', 'AnnualHouseholdIncome', 'AnnualFamilyIncome', 'FamilyIncomePovertyRatio']
demographics_short2.head()

,SEQN,Gender,Age,Race,CountryOfBirth,CitizenshipStatus,FamilySize,AnnualHouseholdIncome,AnnualFamilyIncome,FamilyIncomePovertyRatio
0,93703.0,2.0,2.0,5.0,1.0,1.0,5.0,15.0,15.0,5.00
1,93704.0,1.0,2.0,3.0,1.0,1.0,4.0,15.0,15.0,5.00
2,93705.0,2.0,66.0,4.0,1.0,1.0,1.0,3.0,3.0,0.82
3,93706.0,1.0,18.0,5.0,1.0,1.0,5.0,NaN,NaN,NaN
4,93707.0,1.0,13.0,5.0,1.0,1.0,7.0,10.0,10.0,1.88


Now we have an acceptable amount of data from this dataset, we can move on to the next one

#### Medical Conditions Dataset

For the medical conditions dataset, we are only interested in the column MCQ160c - Ever told you had coronary heart disease. This is our class column, which is asking respondents if a doctor or other health professional ever told them that they had coronary heart disease, and the SEQN to allow us to merge this dataset with all the others. So we will isolate these two columns, and then check for null values

In [120]:
medical_conditions_short = medical_conditions[['SEQN', 'MCQ160C']]
medical_conditions_short.columns = ['SEQN', 'hasHeartDisease']
print('Total number of respondents')
print(len(medical_conditions_short))
print('% NaN occurrences in Columns:')
print(100 * medical_conditions_short.isnull().sum(axis = 0)/len(medical_conditions_short))

Total number of respondents
8897
% NaN occurrences in Columns:
SEQN                0.000000
hasHeartDisease    37.405867
dtype: float64


From this we can see that around 37% of respondents we don't have an answer for, which is quite a high number, however our dataset is very large so we should be able to remove these columns later and still have enough data to perform our analysis

In [121]:
medical_conditions_short = medical_conditions[['SEQN', 'MCQ160C']]
medical_conditions_short.columns = ['SEQN', 'hasHeartDisease']
print(medical_conditions_short.head())

      SEQN  hasHeartDisease
0  93703.0              NaN
1  93704.0              NaN
2  93705.0              2.0
3  93706.0              NaN
4  93707.0              NaN
3328
8897


# Merging

The next step is to merge together the datasets on Respondent Sequence number.

First we take a look at the columns in each dataset to ensure that they all have a Respondent Sequence Number column

In [126]:
print(body_measures_short.info())
print(blood_pressure_short.info())
print(demographics_short2.info())
print(medical_conditions_short.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8704 entries, 0 to 8703
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SEQN    8704 non-null   float64
 1   Weight  8580 non-null   float64
 2   Height  8016 non-null   float64
 3   BMI     8005 non-null   float64
dtypes: float64(4)
memory usage: 272.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8704 entries, 0 to 8703
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SEQN            8704 non-null   float64
 1   diastolic_avg   6077 non-null   float64
 2   systolic_avg    6077 non-null   float64
 3   pulse_rate_BPM  6742 non-null   float64
dtypes: float64(4)
memory usage: 272.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9254 entries, 0 to 9253
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    ---------

Since they all have the SEQN column, we are safe to merge on this column

In [129]:
joined_dataset = pd.merge(body_measures_short, blood_pressure_short, on='SEQN')
joined_dataset = pd.merge(joined_dataset, demographics_short2, on='SEQN')
joined_dataset = pd.merge(joined_dataset, medical_conditions_short, on='SEQN')
print(joined_dataset.head())
print(joined_dataset.info())

      SEQN  Weight  Height   BMI  diastolic_avg  systolic_avg  pulse_rate_BPM  \
0  93703.0    13.7    88.6  17.5            NaN           NaN             NaN   
1  93704.0    13.9    94.2  15.7            NaN           NaN             NaN   
2  93705.0    79.5   158.3  31.7            NaN           NaN            52.0   
3  93706.0    66.3   175.7  21.5      73.333333    111.333333            82.0   
4  93707.0    45.4   158.4  18.1      47.333333    128.000000           100.0   

   Gender   Age  Race  CountryOfBirth  CitizenshipStatus  FamilySize  \
0     2.0   2.0   5.0             1.0                1.0         5.0   
1     1.0   2.0   3.0             1.0                1.0         4.0   
2     2.0  66.0   4.0             1.0                1.0         1.0   
3     1.0  18.0   5.0             1.0                1.0         5.0   
4     1.0  13.0   5.0             1.0                1.0         7.0   

   AnnualHouseholdIncome  AnnualFamilyIncome  FamilyIncomePovertyRatio  \
0     

The first thing we can do is check the amounts of each class in the dataset

In [133]:
print(joined_dataset['hasHeartDisease'].value_counts())

2.0    5006
1.0     243
9.0      16
Name: hasHeartDisease, dtype: int64
2.0    0.598374
1.0    0.029046
9.0    0.001913
Name: hasHeartDisease, dtype: float64


According to the documentation, a 2 indicates a positive result, 1 indicates a negative result, and 9 means the person doesn't know. We will drop these entries as this is a binary classification, and there are only 16 of these rows. Note that there are a large amount of entries where there is no data for this column, so we will have to drop those entries as well, as they do not have a class associated with them. 

In [140]:
joined_dataset = joined_dataset.loc[joined_dataset['hasHeartDisease'].isin([2, 1])]
print(len(joined_dataset))

5249


Now that we have removed that data, looking at the positive and negative results, we can see that this is a heavily imbalanced dataset, because there are roughly 20 times as many people who don't have heart disease as those who do. This is to be expected, as the survey is of the whole country, and according to the CDC about 6.7% of people have Coronary Heart Disease. (https://www.cdc.gov/heartdisease/facts.htm). Nevertheless, this makes our classification harder, as we have to resolve this imbalance. First we will clean up the data some more, then from our clean data will resolve this imbalance

In [141]:
joined_dataset = joined_dataset.dropna()
print(joined_dataset['hasHeartDisease'].value_counts())

2.0    3671
1.0     177
Name: hasHeartDisease, dtype: int64
